<a href="https://colab.research.google.com/github/GauravAero/Script_Writer/blob/GauravAero-patch-1/Script_Writer_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script Writer Using CrewAI


In this notebook, we demonstrate how to build and collaborating agentic AI system using **CrewAI**. The system integrates multiple agents, each with a specific role, to:
1. Gather current information from the internet about given topic.
2. Make a conversational script out of the information.

---
### Workflow Overview
1. Define tools to support the agents, such as web search capabilities.
2. Create two agents for data gathering and script creation.
3. Define tasks corresponding to these agents.
4. Set up a crew that executes the tasks sequentially.
5. Run the crew to achieve the desired outcome.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Define Tools

In [ ]:
## initial setup for Colab.
## if you are using this in local jupyter notebook, skip this cell
!pip install crewai
!pip install crewai-tools
!pip install duckduckgo-search
!pip install langchain-community
!pip install litellm

In [ ]:
from crewai import Agent, Task
from crewai.tools import tool
from crewai.llm import LLM
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime
import os

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)


### Step 2: Setup your API key for LLM and Define the Agents

In [ ]:
from google.colab import userdata

# Retrieve your API key from Colab secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('Gemini_key')

# Use CrewAI's built-in LLM configuration (Recommended)
from crewai import LLM

# Configure Gemini LLM directly through CrewAI
gemini_llm = LLM(
    model="gemini/gemini-2.5-flash",  # Use gemini/ prefix for CrewAI
    api_key=os.environ["GOOGLE_API_KEY"]
)

In [ ]:
from crewai import Agent
#from langchain_community.tools.exa.tool import ExaSearchTool

## Agent designed to gather information about a topic from internet.
content_explorer = Agent (
    role = 'content explorer',
    goal = 'Gather and provide latest information about the topic from internet',
    llm = gemini_llm,
    # llm = 'groq/gemma2-9b-it',
    # llm = 'openrouter/deepseek/deepseek-chat-v3-0324:free',
    # llm = 'gpt-4o',
    verbose = True,
    backstory = ('You are an expert researcher, who can gather detailed information about a topic.\
                  Gather at least 10 information. Consider you are on : '+Today),
    #tools = [Exa_Search_Tool],
    cache = True,
    max_iter = 5
)

## Creat an Agent to write a script from the info gathered by content_explorer.
## Back story sets the context for the agent.

Script_Writer = Agent (
    role = 'Script Writer',
    goal = 'With the details given to you create an interesting conversational script out of it',
    #llm = 'openrouter/deepseek/deepseek-chat-v3-0324:free',(Any other LLM can be used)
    llm = gemini_llm,
    verbose = True,
    backstory = ('You are an expert in Shakesperean literature. You are very good in creating conversations with the given chain of information.\
                 Tell as a script in 200 words.\
                 Consider you are on '+Today),
)

os.environ['OTEL_SDK_DISABLED'] = "True"


### Step 3: Define the Tasks

In [ ]:
from crewai import Task

# Task to gather Latest information
get_details = Task (
                      description = "Get latest, trending, interesting information and news about {topic}",
                      expected_output = "Latest news, interesting information and trivia about {topic}",
                      agent = content_explorer
                    )

## Task to create script.
create_a_script = Task (
                      description = "Considering the given details in time order make an interesting conversation",
                      expected_output = "A humorous conversation connecting key details",
                      agent = Script_Writer,
                      context = [get_details],
                      output_file = 'Script.txt'
                    )



### Step 4: Set Up the Crew

In [ ]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew (
    agents = [content_explorer, Script_Writer],
    tasks = [get_details, create_a_script],
    verbose = True,
    Process = Process.sequential,
    step_callback = timestamp
)

### Step 5: Run the Crew and Observe Results

In [ ]:
# Run the crew with a specific topic
result = crew.kickoff (inputs={'topic' : "Iran Isral conflict"})



---
### Conclusion
This workflow demonstrates how to integrate multiple agents to perform tasks sequentially in a context. It showcases the power of CrewAI for building AI-driven collaborative solutions.
